In [1]:
# Import dependencies
import pandas as pd
import os

In [2]:
# Ensure file permissions are correct to access the files
# True value should be returned 
file_path = "Resources/202301-citibike-tripdata.csv"
print(os.access(file_path, os.R_OK))

True


In [3]:
# Read in files to be cleaned
df1 = pd.read_csv("Resources/202301-citibike-tripdata.csv", low_memory=False)
df2 = pd.read_csv("Resources/202304-citibike-tripdata.csv", low_memory=False)
df3 = pd.read_csv("Resources/202307-citibike-tripdata.csv", low_memory=False)

In [5]:
# Create a new DataFrame to include all csv files
bike_data = [df1, df2, df3]

# Drop any duplicate rows
bike_data_df = pd.concat(bike_data).drop_duplicates()

In [6]:
# Confirm the number of rows in the DataFrame
len(bike_data_df)

8485807

In [13]:
# Display the DataFrame
bike_data_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,4A86C1475DCCADA0,classic_bike,2023-01-26 10:53:44,2023-01-26 11:05:17,E 53 St & Madison Ave,6659.03,Columbus Ave & W 72 St,7175.05,40.759724,-73.973664,40.777057,-73.978985,member
1,AE02E1FF7E264874,classic_bike,2023-01-04 11:51:54,2023-01-04 12:07:09,Halsey St & Ralph Ave,4334.02,Clinton Ave & Myrtle Ave,4651.02,40.684970,-73.922755,40.693261,-73.968896,casual
2,D3F9A2A71AD244C6,classic_bike,2023-01-04 17:26:48,2023-01-04 17:43:29,Bank St & Hudson St,5922.08,W 52 St & 11 Ave,6926.01,40.736566,-74.006092,40.767272,-73.993929,casual
3,3D48C6F33AFEF329,classic_bike,2023-01-04 08:31:11,2023-01-04 08:42:12,E 53 St & Madison Ave,6659.03,W 45 St & 8 Ave,6676.02,40.759711,-73.974023,40.759291,-73.988597,member
4,4865926C3D97BA01,electric_bike,2023-01-03 22:15:34,2023-01-03 22:30:35,Frederick Douglass Blvd & W 112 St,7631.23,Central Park S & 6 Ave,6876.04,40.801694,-73.957145,40.765909,-73.976342,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3776251,8EB48FBE46A673DF,classic_bike,2023-07-28 19:11:45,2023-07-28 19:38:24,S 4 St & Rodney St,5156.05,Irving Ave & Halsey St,4695.04,40.709340,-73.956080,40.694670,-73.906630,member
3776252,8B66692B4C236E00,classic_bike,2023-07-11 20:09:42,2023-07-11 20:38:01,Madison Ave & E 120 St,7667.04,Intervale Ave & Westchester Ave,7948.10,40.803026,-73.942863,40.821967,-73.896752,member
3776253,86E2D896051F1420,classic_bike,2023-07-25 21:05:48,2023-07-25 21:10:15,Windsor Pl & Howard Pl,3579.04,14 St & 5 Ave,3771.06,40.659647,-73.980421,40.666287,-73.988951,member
3776254,556B12591EC0C385,classic_bike,2023-07-15 13:15:14,2023-07-15 13:25:55,DeKalb Ave & Vanderbilt Ave,4461.04,Broadway & Kent Ave,5134.09,40.689412,-73.968847,40.710888,-73.968329,member


In [15]:
# Gather station data to strip it from the citibike file
# Gather all starting stations 
# Drop all duplicate rows to get the unique starting stations
stations_df = bike_data_df[["start_station_id","start_station_name","start_lat","start_lng"]].drop_duplicates("start_station_id")

# Rename columns
stations_df.columns = ["station_id","station_name","lat","lng"]

# Gather all ending stations
# # Drop all duplicate rows to get the unique ending stations
end_stations_df = bike_data_df[["end_station_id","end_station_name","end_lat","end_lng"]].drop_duplicates("end_station_id")

# Rename columns
end_stations_df.columns = ["station_id","station_name","lat","lng"]

In [17]:
# Combine the above two DataFrames into one df and remove any duplicate rows and null value rows
stations_df = pd.concat([stations_df,end_stations_df]).drop_duplicates("station_id")
stations_df = stations_df.dropna()
stations_df.to_csv("Resources/cleaned_citibike_stations.csv", index=False)

In [21]:
stations_df.head()

,station_id,station_name,lat,lng
0,6659.03,E 53 St & Madison Ave,40.759724,-73.973664
1,4334.02,Halsey St & Ralph Ave,40.684970,-73.922755
2,5922.08,Bank St & Hudson St,40.736566,-74.006092
4,7631.23,Frederick Douglass Blvd & W 112 St,40.801694,-73.957145
6,5626.15,1 Ave & E 6 St,40.726331,-73.986169


In [33]:
# Remove all station_id information from the bike_data_df but leave the start and end station_ids
citibikes_df = bike_data_df.drop(columns=["start_station_name",
                                          "end_station_name",
                                          "start_lat",
                                          "start_lng",
                                          "end_lat",
                                          "end_lng"])
citibikes_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual
0,4A86C1475DCCADA0,classic_bike,2023-01-26 10:53:44,2023-01-26 11:05:17,6659.03,7175.05,member
1,AE02E1FF7E264874,classic_bike,2023-01-04 11:51:54,2023-01-04 12:07:09,4334.02,4651.02,casual
2,D3F9A2A71AD244C6,classic_bike,2023-01-04 17:26:48,2023-01-04 17:43:29,5922.08,6926.01,casual
3,3D48C6F33AFEF329,classic_bike,2023-01-04 08:31:11,2023-01-04 08:42:12,6659.03,6676.02,member
4,4865926C3D97BA01,electric_bike,2023-01-03 22:15:34,2023-01-03 22:30:35,7631.23,6876.04,casual


In [45]:
# Convert start_at and ended_at to date time 
citibikes_df["started_at"] = pd.to_datetime(citibikes_df["started_at"])
citibikes_df["ended_at"] = pd.to_datetime(citibikes_df["ended_at"])

In [46]:
# Confirm the start and end times have been changed to datetime
citibikes_df.dtypes

ride_id                     object
rideable_type               object
started_at          datetime64[ns]
ended_at            datetime64[ns]
start_station_id            object
end_station_id              object
member_casual               object
dtype: object

In [47]:
# Calculate the time spent on the citi bikes
# Add the column to citibikes_df
citibikes_df["trip_length"] = (citibikes_df["ended_at"] - citibikes_df["started_at"]).dt.total_seconds() / 60

citibikes_df

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual,trip_length
0,4A86C1475DCCADA0,classic_bike,2023-01-26 10:53:44,2023-01-26 11:05:17,6659.03,7175.05,member,11.550000
1,AE02E1FF7E264874,classic_bike,2023-01-04 11:51:54,2023-01-04 12:07:09,4334.02,4651.02,casual,15.250000
2,D3F9A2A71AD244C6,classic_bike,2023-01-04 17:26:48,2023-01-04 17:43:29,5922.08,6926.01,casual,16.683333
3,3D48C6F33AFEF329,classic_bike,2023-01-04 08:31:11,2023-01-04 08:42:12,6659.03,6676.02,member,11.016667
4,4865926C3D97BA01,electric_bike,2023-01-03 22:15:34,2023-01-03 22:30:35,7631.23,6876.04,casual,15.016667
...,...,...,...,...,...,...,...,...
3776251,8EB48FBE46A673DF,classic_bike,2023-07-28 19:11:45,2023-07-28 19:38:24,5156.05,4695.04,member,26.650000
3776252,8B66692B4C236E00,classic_bike,2023-07-11 20:09:42,2023-07-11 20:38:01,7667.04,7948.10,member,28.316667
3776253,86E2D896051F1420,classic_bike,2023-07-25 21:05:48,2023-07-25 21:10:15,3579.04,3771.06,member,4.450000
3776254,556B12591EC0C385,classic_bike,2023-07-15 13:15:14,2023-07-15 13:25:55,4461.04,5134.09,member,10.683333


In [48]:
# Drop any rows with total_time_spent_minutes less than 1 minute
citibikes_df = citibikes_df[citibikes_df["trip_length"] >= 1]

In [49]:
# Convert the DataFrame to a csv 
citibikes_df.to_csv("Resources/citibikes_cleaned_data.csv")